In [8]:
import heapq

# --- 1. Definición del Problema y Datos Iniciales ---

# Número de usuarios
NUM_USERS = 9

# Simulación de datos iniciales para 4 propuestas en el Frente de Pareto
# Cada propuesta es un diccionario con 'apoyos' y 'rechazos'
# Los usuarios se representan como 'U1', 'U2', etc.
# Los rechazos se representan como 'R1', 'R2', etc.

# Ejemplo de datos iniciales (esto debería venir de tu sistema real)
initial_proposals_data = {
    "Prop1": {"apoyos": {"U1", "U2", "U3"}, "rechazos": {"R4", "R5"}},
    "Prop2": {"apoyos": {"U2", "U4", "U6"}, "rechazos": {"R1", "R3"}},
    "Prop3": {"apoyos": {"U1", "U5", "U7"}, "rechazos": {"R2", "R6"}},
    "Prop4": {"apoyos": {"U3", "U8", "U9"}, "rechazos": {"R7", "R9"}},
}

# Mapeo de usuarios y rechazos a un formato numérico para facilitar el cálculo
# U1 -> 0, U2 -> 1, ..., R1 -> 0, R2 -> 1, ...
user_map = {f"U{i+1}": i for i in range(NUM_USERS)}

# --- 2. Definición del Estado y Acciones ---

# Un estado se define como una tupla: (frozenset_apoyos, frozenset_rechazos)
# Usamos frozenset para que sea inmutable y hashable (necesario para conjuntos en diccionarios/sets)

# Las acciones (enmiendas evolutivas) transforman un estado en otro.
# Para este ejemplo, simplificaremos las acciones a:
# 1. Cambiar el apoyo de un usuario a rechazo.
# 2. Cambiar el rechazo de un usuario a apoyo.
# 3. Cambiar el estado de un usuario (apoyo/rechazo) a NS/NC (No Sabe/No Contesta).
# 4. Cambiar el estado de un usuario de NS/NC a apoyo/rechazo.

# Costos de las acciones (ejemplo, estos deben ser definidos por el usuario)
# Costo de cambiar de apoyo a rechazo o viceversa
COST_OPINION_CHANGE = 1.0
# Costo de cambiar a NS/NC o de NS/NC a apoyo/rechazo
COST_TO_NSNC = 0.5

# --- 3. Funciones de Costo y Heurística ---

# Función para calcular la "cercanía" entre dos propuestas (estados)
# Aquí se implementaría la lógica de cercanía entre propuestas.
# Para este ejemplo, una simplificación: cuántos usuarios tienen el mismo estado (apoyo/rechazo)
def calculate_closeness_cost(state1, state2, weights):
    apoyos1, rechazos1 = state1
    apoyos2, rechazos2 = state2

    common_apoyos = len(apoyos1.intersection(apoyos2))
    common_rechazos = len(rechazos1.intersection(rechazos2))

    # Cuanto más comunes, menor el costo de cercanía
    # Esto es una simplificación. La cercanía real debería ser una función inversa de la distancia.
    # Aquí lo interpretamos como un costo de transición: cuanto más diferentes, mayor el costo.
    diff_apoyos = len(apoyos1.symmetric_difference(apoyos2))
    diff_rechazos = len(rechazos1.symmetric_difference(rechazos2))

    # Un costo simple basado en las diferencias
    cost = (diff_apoyos * weights["apoyo_diff_cost"] +
            diff_rechazos * weights["rechazo_diff_cost"])
    return cost

# Función para calcular el costo de una "enmienda evolutiva" (transición entre estados)
# Esto es el costo de mover un usuario de un estado a otro.
def calculate_amendment_cost(user_id, old_sentiment, new_sentiment, weights):
    # old_sentiment y new_sentiment son el estado del usuario (apoyo, rechazo, NS/NC)
    # Por ejemplo, si old_sentiment es 'apoyo' y new_sentiment es 'rechazo', el costo es weights["apoyo_to_rechazo"]
    # Si old_sentiment es 'apoyo' y new_sentiment es 'NS/NC', el costo es weights["to_nsnc"]
    # Esta función debe ser mucho más detallada según tus definiciones de costo.

    cost = 0.0
    if old_sentiment == "apoyo" and new_sentiment == "rechazo":
        cost = weights["apoyo_to_rechazo"]
    elif old_sentiment == "rechazo" and new_sentiment == "apoyo":
        cost = weights["rechazo_to_apoyo"]
    elif (old_sentiment == "apoyo" or old_sentiment == "rechazo") and new_sentiment == "NS/NC":
        cost = weights["to_nsnc"]
    elif old_sentiment == "NS/NC" and (new_sentiment == "apoyo" or new_sentiment == "rechazo"):
        cost = weights["from_nsnc"]

    return cost

# Heurística (para A*): estimación del costo restante hasta el objetivo
# Para este problema, una heurística simple podría ser el número de usuarios que aún no apoyan la propuesta final.
def heuristic(state, goal_state):
    apoyos_current, rechazos_current = state
    apoyos_goal, rechazos_goal = goal_state

    # Número de usuarios que no están en el estado deseado
    missing_apoyos = len(apoyos_goal - apoyos_current)
    extra_apoyos = len(apoyos_current - apoyos_goal)
    missing_rechazos = len(rechazos_goal - rechazos_current)
    extra_rechazos = len(rechazos_current - rechazos_goal)

    # Una heurística simple: suma de diferencias
    return (missing_apoyos + extra_apoyos + missing_rechazos + extra_rechazos) * 0.5 # Multiplicador para subestimar

# --- 4. Algoritmo de Búsqueda (A*) ---

def find_consensus_path(start_state, goal_state, initial_proposals_data, weights):
    # open_set es una cola de prioridad de tuplas (costo_f, costo_g, estado, camino)
    open_set = []
    heapq.heappush(open_set, (0, 0, start_state, [start_state])) # (f_cost, g_cost, state, path)

    # g_costs almacena el costo del camino más barato conocido hasta un estado
    g_costs = {start_state: 0}

    # closed_set para evitar ciclos y re-procesar estados
    closed_set = set()

    while open_set:
        f_cost, g_cost, current_state, path = heapq.heappop(open_set)

        if current_state == goal_state:
            return path, g_cost

        if current_state in closed_set:
            continue
        closed_set.add(current_state)

        # Generar estados vecinos (enmiendas evolutivas)
        # Para cada usuario, considera cambiar su estado (apoyo/rechazo/NSNC)
        current_apoyos, current_rechazos = current_state

        for user_idx in range(NUM_USERS):
            user_str = f"U{user_idx+1}"

            # Determinar el estado actual del usuario en esta propuesta
            user_current_sentiment = "NS/NC"
            if user_str in current_apoyos:
                user_current_sentiment = "apoyo"
            elif f"R{user_idx+1}" in current_rechazos:
                user_current_sentiment = "rechazo"

            # Posibles nuevos estados para el usuario
            possible_new_sentiments = [s for s in ["apoyo", "rechazo", "NS/NC"] if s != user_current_sentiment]

            for new_sentiment in possible_new_sentiments:
                next_apoyos = set(current_apoyos)
                next_rechazos = set(current_rechazos)

                # Aplicar el cambio de sentimiento del usuario
                if user_current_sentiment == "apoyo":
                    next_apoyos.discard(user_str)
                elif user_current_sentiment == "rechazo":
                    next_rechazos.discard(f"R{user_idx+1}")

                if new_sentiment == "apoyo":
                    next_apoyos.add(user_str)
                elif new_sentiment == "rechazo":
                    next_rechazos.add(f"R{user_idx+1}")
                # Si new_sentiment es NS/NC, no se añade a ningún conjunto

                next_state = (frozenset(next_apoyos), frozenset(next_rechazos))

                # Calcular el costo de la enmienda
                amendment_cost = calculate_amendment_cost(user_idx, user_current_sentiment, new_sentiment, weights)

                # Aquí es donde la "cercanía entre propuestas" y otras métricas
                # deberían influir en el costo de transición entre estados.
                # Por simplicidad, aquí solo consideramos el costo de la enmienda.
                # Podrías añadir un costo adicional si el next_state se aleja mucho de las propuestas iniciales.
                # Por ejemplo, si el next_state es muy diferente de las propuestas iniciales del Frente de Pareto,
                # se podría añadir un costo extra para desincentivar esos caminos.

                new_g_cost = g_cost + amendment_cost # + calculate_closeness_cost(current_state, next_state, weights)

                if next_state not in g_costs or new_g_cost < g_costs[next_state]:
                    g_costs[next_state] = new_g_cost
                    h_cost = heuristic(next_state, goal_state)
                    f_cost = new_g_cost + h_cost
                    heapq.heappush(open_set, (f_cost, new_g_cost, next_state, path + [next_state]))

    return None, float("inf") # No se encontró un camino

# --- 5. Configuración y Ejecución ---

if __name__ == "__main__":
    # Definir el estado inicial (por ejemplo, la primera propuesta del Frente de Pareto)
    # Convertir los sets a frozensets para que el estado sea hashable
    start_proposal_name = list(initial_proposals_data.keys())[0]
    start_apoyos = frozenset(initial_proposals_data[start_proposal_name]["apoyos"])
    start_rechazos = frozenset(initial_proposals_data[start_proposal_name]["rechazos"])
    start_state = (start_apoyos, start_rechazos)

    # Definir el estado objetivo (todos los usuarios apoyan, ninguno rechaza)
    goal_apoyos = frozenset([f"U{i+1}" for i in range(NUM_USERS)])
    goal_rechazos = frozenset()
    goal_state = (goal_apoyos, goal_rechazos)

    # Definir los pesos para los costos (estos son ejemplos, ajusta según tu lógica)
    # La "cercanía" entre propuestas y el "cambio de opinión" se reflejan aquí.
    # Por ejemplo, un costo alto para cambiar de rechazo a apoyo.
    weights = {
        "apoyo_to_rechazo": 2.0,
        "rechazo_to_apoyo": 5.0, # Más costoso cambiar un rechazo a apoyo
        "to_nsnc": 1.0,
        "from_nsnc": 1.5,
        "apoyo_diff_cost": 0.5, # Costo por diferencia en apoyos al calcular cercanía
        "rechazo_diff_cost": 0.5, # Costo por diferencia en rechazos al calcular cercanía
    }

    print(f"Buscando camino de consenso desde {start_state} hasta {goal_state}\n")
    path, total_cost = find_consensus_path(start_state, goal_state, initial_proposals_data, weights)

    if path:
        print(f"Camino encontrado con costo total: {total_cost:.2f}\n")
        for i, state in enumerate(path):
            apoyos, rechazos = state
            print(f"Paso {i}: Apoyos: {list(apoyos)}, Rechazos: {list(rechazos)}")
    else:
        print("No se encontró un camino hacia el consenso con los parámetros dados.")






Buscando camino de consenso desde (frozenset({'U2', 'U1', 'U3'}), frozenset({'R4', 'R5'})) hasta (frozenset({'U2', 'U6', 'U9', 'U3', 'U4', 'U1', 'U8', 'U5', 'U7'}), frozenset())

Camino encontrado con costo total: 11.00

Paso 0: Apoyos: ['U2', 'U1', 'U3'], Rechazos: ['R4', 'R5']
Paso 1: Apoyos: ['U2', 'U1', 'U3'], Rechazos: ['R5']
Paso 2: Apoyos: ['U2', 'U1', 'U3'], Rechazos: []
Paso 3: Apoyos: ['U2', 'U8', 'U1', 'U3'], Rechazos: []
Paso 4: Apoyos: ['U2', 'U8', 'U3', 'U5', 'U1'], Rechazos: []
Paso 5: Apoyos: ['U2', 'U8', 'U9', 'U3', 'U5', 'U1'], Rechazos: []
Paso 6: Apoyos: ['U2', 'U8', 'U9', 'U3', 'U5', 'U4', 'U1'], Rechazos: []
Paso 7: Apoyos: ['U2', 'U9', 'U3', 'U4', 'U1', 'U8', 'U5', 'U7'], Rechazos: []
Paso 8: Apoyos: ['U2', 'U6', 'U9', 'U3', 'U4', 'U1', 'U8', 'U5', 'U7'], Rechazos: []
